In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import random
import mysql.connector
from datetime import datetime

user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko",
    "Mozilla/5.0 (iPad; CPU OS 13_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 13_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0",
    "Mozilla/5.0 (Linux; Android 10; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.96 Mobile Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/83.0.478.37",
    "Mozilla/5.0 (Linux; Android 10; SM-G973F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.96 Mobile Safari/537.36"
]
def create_driver_with_user_agent(user_agent):
    options = Options()
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
    options.add_argument(f'user-agent={user_agent}')
    return webdriver.Chrome(options=options)

def go_to_post_page(post_url):
    driver = create_driver_with_user_agent(random.choice(user_agents))
    driver.get(post_url)
    time.sleep(3)
    try:
        cookies = "/html/body/div[4]/div/div/div[2]/div/div/div/div[2]/div/div/div/div[2]/div/div[2]"
        cookies2 = "/html/body/div[3]/div/div/div[2]/div/div/div/div[2]/div/div/div/div[2]/div/div[2]"
        try:
            cookies_button = driver.find_element(By.XPATH, cookies)
        except:
            cookies_button = driver.find_element(By.XPATH, cookies2)
        cookies_button.click()
        time.sleep(2)
    except:
        delete_unavailable(post_url)
        driver.close()
        return None
    return driver

def connect_to_db():
    db_params = {
        'host': '192.168.100.200',
        'user': 'ecorradini',
        'password': 'birillo',
        'database': 'threads'
    }
    db_connection = mysql.connector.connect(**db_params)
    db_cursor = db_connection.cursor()
    return db_connection,db_cursor

def fetch_posts():
    db_connection,db_cursor = connect_to_db()
    query = """
        SELECT p.post_url
        FROM post p
        WHERE NOT EXISTS (
            SELECT 1
            FROM post_data pd
            WHERE pd.url = p.post_url
        );
    """
    db_cursor.execute(query)
    return db_cursor.fetchall()

def update_image(post_url, image_url):
    db_connection,db_cursor = connect_to_db()
    update_query = "UPDATE post SET image_url = %s WHERE post_url = %s"
    db_cursor.execute(update_query, (image_url, post_url))
    db_connection.commit()
    db_cursor.close()
    db_connection.close()
    
def insert_comment(comment_url, post_url, user, image, caption, c_time):
    db_params = {
        'host': '192.168.100.200',  # Or your database host
        'user': 'ecorradini',   # Your database username
        'password': 'birillo',
        'database': 'threads'
    }
    db_connection = mysql.connector.connect(**db_params)
    db_cursor = db_connection.cursor()
    
    if caption is not None:
        c = caption.replace('"','\"').replace("'","\'")
    else:
        c = caption
    insert_query = "INSERT INTO comment (url, post, user, time, caption, image) VALUES (%s, %s, %s, %s, %s, %s)"
    timestamp = datetime.strptime(c_time, "%Y-%m-%dT%H:%M:%S.%fZ").timestamp()
    formatted_time = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
    data = (comment_url,post_url,user,formatted_time,c,image)
    db_cursor.execute(insert_query, data)
    db_connection.commit()
    
    db_cursor.close()
    db_connection.close()
    
def insert_post_data(url, p_time, likes):
    db_params = {
        'host': '192.168.100.200',  # Or your database host
        'user': 'ecorradini',   # Your database username
        'password': 'birillo',
        'database': 'threads'
    }
    db_connection = mysql.connector.connect(**db_params)
    db_cursor = db_connection.cursor()
    
    insert_query = "INSERT INTO post_data (url, time, likes) VALUES (%s, %s, %s)"
    timestamp = datetime.strptime(p_time, "%Y-%m-%dT%H:%M:%S.%fZ").timestamp()
    formatted_time = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
    data = (url,formatted_time,likes)
    db_cursor.execute(insert_query, data)
    db_connection.commit()
    
    db_cursor.close()
    db_connection.close()
    
def delete_unavailable(post_url):
    db_params = {
        'host': '192.168.100.200',  # Or your database host
        'user': 'ecorradini',   # Your database username
        'password': 'birillo',
        'database': 'threads'
    }
    db_connection = mysql.connector.connect(**db_params)
    db_cursor = db_connection.cursor()
    
    delete_query1 = f"DELETE FROM images WHERE post_url='{post_url}'"
    db_cursor.execute(delete_query1)
    delete_query2 = f"DELETE FROM post WHERE post_url='{post_url}'"
    db_cursor.execute(delete_query2)
    db_connection.commit()
    
    db_cursor.close()
    db_connection.close()

In [2]:
base_path = "/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]"
post_path = base_path + "/div[1]/div/div/div"
comments_path = base_path + "/div[position() > 1 and position() < last() - 2]"

def get_video_url(driver):
    video_path = post_path + "/div/div[3]/div/div[2]/div/div/div/div/div/div/div/div/div/div/video"
    try:
        video = driver.find_element(By.XPATH,video_path)
        video_url = video.get_attribute('src')
        return video_url
    except:
        return None

def get_image_url(driver):
    image_path = post_path + "/div[2]/div[3]/div/div[2]/div/div/picture/img"
    try:
        picture = driver.find_element(By.XPATH,image_path)
        image_url = picture.get_attribute('src')
        return image_url
    except:
        return None
    
def get_time(driver):
    time_path = post_path + "/div[2]/div[2]/div/div[2]/span/a/time"
    try:
        time_p = driver.find_element(By.XPATH,time_path)
        post_time = time_p.get_attribute("datetime")
        return post_time
    except:
        return None

def get_likes(driver):
    likes_path = post_path + "/div[2]/div[4]/div/span/span/div/div"
    try:
        return int(driver.find_element(By.XPATH,likes_path).text.replace(",", ""))
    except Exception as e:
        likes_path2 = post_path + "/div[2]/div[4]/div/a/span/span/div/div"
        try:
            return int(driver.find_element(By.XPATH,likes_path2).text.replace(",", ""))
        except:
            return 0
    
def parse_comments(driver):
    comment_base_path = "./div/div[1]/div/div[2]"
    comment_url_path = comment_base_path + "/div[2]/div/div[2]/span/a"
    user_path = comment_base_path + "/div[2]/div/div[1]/span/div/div/a"
    time_path = comment_base_path + "/div[2]/div/div[2]/span/a/time"
    contents_path = comment_base_path + "/div[3]/div"
    contents_path2 = comment_base_path + "/div[4]/div"
    comment_url = driver.find_element(By.XPATH,comment_url_path).get_attribute('href')
    user = driver.find_element(By.XPATH,user_path)
    user_url = user.get_attribute('href')
    time = driver.find_element(By.XPATH,time_path)
    comment_time = time.get_attribute("datetime")
    contents = driver.find_elements(By.XPATH,contents_path)
    if len(contents) > 0:
        caption,image = parse_comment_contents(contents)
    else:
        contents2 = driver.find_elements(By.XPATH,contents_path2)
        caption,image = parse_comment_contents(contents2)
    return comment_url,user_url,comment_time,caption,image
    
def parse_comment_contents(comment_contents):
    #caption_path = "./div[1]"
    caption = comment_contents[0].text
    image_path = "./div/div/picture/img"
    try:
        image = comment_contents[1].find_element(By.XPATH,image_path).get_attribute('src')
    except:
        image = ""
    return caption,image

In [3]:
while True:
    posts = [x[0] for x in fetch_posts()]
    if len(posts) > 0:
        post = posts[0]
        driver = go_to_post_page(post)
        if driver is not None:
            image = get_image_url(driver)
            video = get_video_url(driver)
            if image:
                update_image(post, image)
            elif video:
                update_image(post, video)
            p_time = get_time(driver)
            likes = get_likes(driver)
            insert_post_data(post,p_time,likes)
            comments = driver.find_elements(By.XPATH,comments_path)
            for comment in comments:
                try:
                    comment_url,comment_user,comment_time,comment_caption,comment_image = parse_comments(comment)
                    insert_comment(comment_url,post,comment_user, comment_image, comment_caption,comment_time)
                except:
                    continue
            driver.close()

KeyboardInterrupt: 

In [70]:
posts = [x[0] for x in fetch_posts()]
driver = go_to_post_page(posts[0])

In [73]:
image = get_image_url(driver)
image

'https://scontent.cdninstagram.com/v/t51.2885-15/419490652_1091811595571112_4052729355745947940_n.jpg?stp=dst-jpg_e35&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi4xNDQweDE4MDAuc2RyIn0&_nc_ht=scontent.cdninstagram.com&_nc_cat=100&_nc_ohc=wz53J5mLEFIAX8hKFe8&edm=APs17CUBAAAA&ccb=7-5&ig_cache_key=MzI4MzIxNDk1OTcyNjEwNzgzOQ%3D%3D.2-ccb7-5&oh=00_AfC96YPmfYlzpbKuqdRuKA8dJ029AEatNYl7UJyo4qxyEA&oe=65AF0E66&_nc_sid=10d13b'

In [37]:
driver = go_to_post_page("posts[5]")

In [64]:
print(get_time(driver))
print(get_likes(driver))

2024-01-09T12:36:40.000Z
None


In [73]:
comments = driver.find_elements(By.XPATH,comments_path)
#print(comments[0].get_attribute("innerHTML"))
print(len(comments))
for comment in comments:
    print(parse_comments(comment))

1
[<selenium.webdriver.remote.webelement.WebElement (session="b47baf00b3878937ff4c378a343ae8e0", element="755B2A35ABE483EA7B16E2F8C4AA8252_element_13")>]
('https://www.threads.net/@tgibb5081/post/C04UKt1Olhd', 'https://www.threads.net/@tgibb5081', '2023-12-15T16:11:40.000Z', 'She looks terrible', '')


In [71]:
parse_comments(comments[0])

[]


IndexError: list index out of range

In [58]:
driver.find_element(By.XPATH,post_path + "/div[2]/div[3]/div/div[2]/div/div/picture").text

''

In [74]:
get_image_url(driver)

'https://scontent.cdninstagram.com/v/t51.2885-15/410351190_1098376414523509_8444690351365330488_n.jpg?stp=dst-jpg_e35&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi4xMTcweDExNjkuc2RyIn0&_nc_ht=scontent.cdninstagram.com&_nc_cat=101&_nc_ohc=y8mZzy0IekoAX-x9fw3&edm=APs17CUBAAAA&ccb=7-5&ig_cache_key=MzI1Nzg1OTA2MzY2OTI3NTg1NA%3D%3D.2-ccb7-5&oh=00_AfB_iXFwYztaFSUhH_T0OVhylto9Dm7V0axv_Yjvyk41tg&oe=6586D0DD&_nc_sid=10d13b'

In [70]:
get_video_url(driver)

'https://scontent.cdninstagram.com/v/t50.2886-16/412493402_759190752914194_5492467403239883235_n.mp4?_nc_ht=scontent.cdninstagram.com&_nc_cat=110&_nc_ohc=s9OjxdSZ2KsAX8g3zLL&edm=APs17CUBAAAA&ccb=7-5&oh=00_AfD8SaA5MNf461F68o-TlxBaWaBGUak7LfXFLVPImkhU5A&oe=65847815&_nc_sid=10d13b'